<a href="https://colab.research.google.com/github/nmningmei/NMA-2021-behavioral-IBL/blob/main/fitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -U pip

In [4]:
!pip cache purge

Files removed: 4


In [5]:
#!pip uninstall numpy
!pip install numpy==1.19.3
!pip install -U mkl
!pip install pymc
!pip install kabuki
!pip install hddm==0.8.0

     |████████████████████████████████| 278.6 MB 35 kB/s 
     |████████████████████████████████| 4.1 MB 34.1 MB/s 
  Attempting uninstall: mkl
    Found existing installation: mkl 2019.0
    Uninstalling mkl-2019.0:
      Successfully uninstalled mkl-2019.0
     |████████████████████████████████| 385 kB 5.0 MB/s 
  Created wheel for pymc: filename=pymc-2.3.8-cp37-cp37m-linux_x86_64.whl size=1353049 sha256=63c12567ef8e680b955d192eca0ac7bce905db2042993af587a7a5e5cafd2134
  Stored in directory: /root/.cache/pip/wheels/3b/9b/1d/2a2ddbe80519db2698745f0988ce50c36ad38aac886a4ee0ca
Successfully built pymc
     |████████████████████████████████| 44 kB 1.8 MB/s 
  Created wheel for kabuki: filename=kabuki-0.6.3-py3-none-any.whl size=36544 sha256=62f7a386d284dc5a955fb484f8dfcfa515aa8d8a3d44ed5eb203cf2128f146e4
  Stored in directory: /root/.cache/pip/wheels/29/cb/6a/02f80c7cde0e973c5ec0f4202aee3ab1b10d84682052d99af5
Successfully built kabuki
     |████████████████████████████████| 601 kB 5.4 MB/s

In [6]:
import hddm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.7/dist-packages/IPython/parallel.py:13: ShimWarning: The `IPython.parallel` package has been deprecated since IPython 4.0. You should import from ipyparallel instead.
  "You should import from ipyparallel instead.", ShimWarning)


In [7]:
!pip install --quiet nma-ibl
import datajoint as dj
dj.config['database.host'] = 'datajoint-public.internationalbrainlab.org'
dj.config['database.user'] = 'nma-ibl-public'
dj.config['database.password'] = 'ibl.pipeline.public.demo'
from nma_ibl import reference, subject, action, acquisition, data, behavior, behavior_analyses


Connecting nma-ibl-public@datajoint-public.internationalbrainlab.org:3306


In [8]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('whitegrid')
sns.set_context('poster')
from matplotlib import pyplot as plt

from tqdm import tqdm

In [9]:
df_nickname = subject.Subject.proj('subject_nickname').fetch(format = 'frame').reset_index()
df_nickname.head()

,subject_uuid,subject_nickname
0,00778394-c956-408d-8a6c-ca3b05a611d5,KS019
1,019a22c1-b944-4494-9e38-0e45ae6697bf,SWC_022
2,01be78e7-8741-4b40-bd64-79ed745431b5,CSHL057
3,02120449-9b19-4276-a434-513886c2fb19,ibl_witten_07
4,030f3e05-9db0-46ba-a6ce-4274ff09b39e,KS031


In [10]:
#EXAMPLE_MOUSE = df_nickname.loc[idx_subject,'subject_nickname']  # Mouse nickname used as an example
CUTOFF_DATE = '2020-03-23'  # Date after which sessions are excluded, previously 30th Nov
STABLE_HW_DATE = '2019-06-10'  # Date after which hardware was deemed stable
EXCLUDED_SESSIONS = ['a9fb578a-9d7d-42b4-8dbc-3b419ce9f424']
def query_subjects(as_dataframe=False, from_list=False, criterion='trained'):
    """
    Query all mice for analysis of behavioral data
    Parameters
    ----------
    as_dataframe:    boolean if true returns a pandas dataframe (default is False)
    from_list:       loads files from list uuids (array of uuids objects)
    criterion:       what criterion by the 30th of November - trained (a and b), biased, ephys
                     (includes ready4ephysrig, ready4delay and ready4recording).  If None,
                     all mice that completed a training session are returned, with date_trained
                     being the date of their first training session.
    """

    # Query all subjects with project ibl_neuropixel_brainwide_01 and get the date at which
    # they reached a given training status
    all_subjects = (subject.Subject * subject.SubjectLab * reference.Lab * subject.SubjectProject
                    & 'subject_project = "ibl_neuropixel_brainwide_01"')
    sessions = acquisition.Session * behavior_analyses.SessionTrainingStatus()
    fields = ('subject_nickname', 'sex', 'subject_birth_date', 'institution_short')

    if criterion is None:
        # Find first session of all mice; date_trained = date of first training session
        subj_query = all_subjects.aggr(
            sessions, *fields, date_trained='min(date(session_start_time))')
    else:  # date_trained = date of first session when criterion was reached
        if criterion == 'trained':
            restriction = 'training_status="trained_1a" OR training_status="trained_1b"'
        elif criterion == 'biased':
            restriction = 'task_protocol LIKE "%biased%"'
        elif criterion == 'ephys':
            restriction = 'training_status LIKE "ready%"'
        else:
            raise ValueError('criterion must be "trained", "biased" or "ephys"')
        subj_query = all_subjects.aggr(
            sessions & restriction, *fields, date_trained='min(date(session_start_time))')

    if from_list is True:
        data_path = os.path.join(datapath(), 'uuids_trained.npy')
        ids = np.load(data_path, allow_pickle=True)
        subj_query = subj_query & [{'subject_uuid': u_id} for u_id in ids]

    # Select subjects that reached criterion before cutoff date
    subjects = (subj_query & 'date_trained <= "%s"' % CUTOFF_DATE)
    if as_dataframe is True:
        subjects = subjects.fetch(format='frame')
        subjects = subjects.sort_values(by=['lab_name']).reset_index()

    return subjects


def query_sessions(task='all', stable=False, as_dataframe=False,
                   force_cutoff=False, criterion='biased'):
    """
    Query all sessions for analysis of behavioral data
    Parameters
    ----------
    task:            string indicating sessions of which task to return, can be trianing or biased
                     default is all
    stable:          boolean if True only return sessions with stable hardware, which means
                     sessions after particular date (default is False)
    as_dataframe:    boolean if True returns a pandas dataframe (default is False)
    force_cutoff:    whether the animal had to reach the criterion by the 30th of Nov. Only
                     applies to biased and ready for ephys criterion
    criterion:       what criterion by the 30th of November - trained (includes
                     a and b), biased, ready (includes ready4ephysrig, ready4delay and
                     ready4recording)
    """

    # Query sessions
    if force_cutoff is True:
        use_subjects = query_subjects(criterion=criterion).proj('subject_uuid')
    else:
        use_subjects = query_subjects().proj('subject_uuid')

    # Query all sessions or only training or biased if required
    if task == 'all':
        sessions = acquisition.Session * use_subjects & 'task_protocol NOT LIKE "%habituation%"'
    elif task == 'training':
        sessions = acquisition.Session * use_subjects & 'task_protocol LIKE "%training%"'
    elif task == 'biased':
        sessions = acquisition.Session * use_subjects & 'task_protocol LIKE "%biased%"'
    elif task == 'ephys':
        sessions = acquisition.Session * use_subjects & 'task_protocol LIKE "%ephys%"'
    else:
        raise ValueError('task must be "all", "training", "biased" or "ephys"')

    # Only use sessions up until the end of December
    sessions = sessions & 'date(session_start_time) <= "%s"' % CUTOFF_DATE

    # Exclude weird sessions
    sessions = sessions & dj.Not([{'session_uuid': u_id} for u_id in EXCLUDED_SESSIONS])

    # If required only output sessions with stable hardware
    if stable is True:
        sessions = sessions & 'date(session_start_time) > "%s"' % STABLE_HW_DATE

    # Transform into pandas Dataframe if requested
    if as_dataframe is True:
        sessions = sessions.fetch(
            order_by='institution_short, subject_nickname, session_start_time', format='frame')
        sessions = sessions.reset_index()

    return sessions
def query_sessions_around_criterion(criterion='trained', days_from_criterion=(2, 0),
                                    as_dataframe=False, force_cutoff=False):
    """
    Query all sessions for analysis of behavioral data
    Parameters
    ----------
    criterion:              string indicating which criterion to use: trained, biased or ephys
    days_from_criterion:    two-element array which indicates which training days around the day
                            the mouse reached criterium to return, e.g. [3, 2] returns three days
                            before criterium reached up until 2 days after (default: [2, 0])
    as_dataframe:           return sessions as a pandas dataframe
    force_cutoff:           whether the animal had to reach the criterion by the 30th of Nov. Only
                            applies to biased and ready for ephys criterion
    Returns
    ---------
    sessions:               The sessions around the criterion day, works in conjunction with
                            any table that has session_start_time as primary key (such as
                            behavior.TrialSet.Trial)
    days:                   The training days around the criterion day. Can be used in conjunction
                            with tables that have session_date as primary key (such as
                            behavior_analyses.BehavioralSummaryByDate)
    """

    # Query all included subjects
    if force_cutoff is True:
        use_subjects = query_subjects(criterion=criterion).proj('subject_uuid')
    else:
        use_subjects = query_subjects().proj('subject_uuid')

    # Query per subject the date at which the criterion is reached
    sessions = acquisition.Session * behavior_analyses.SessionTrainingStatus
    if criterion == 'trained':
        restriction = 'training_status="trained_1a" OR training_status="trained_1b"'
    elif criterion == 'biased':
        restriction = 'task_protocol LIKE "%biased%" AND training_status="trained_1b"'
    elif criterion == 'ephys':
        restriction = 'training_status LIKE "ready%"'
    else:
        raise ValueError('criterion must be "trained", "biased" or "ephys"')

    subj_crit = (subject.Subject * use_subjects).aggr(
        sessions & restriction, 'subject_nickname', date_criterion='min(date(session_start_time))')

    # Query the training day at which criterion is reached
    subj_crit_day = (dj.U('subject_uuid', 'day_of_crit')
                     & (behavior_analyses.BehavioralSummaryByDate * subj_crit
                        & 'session_date=date_criterion').proj(day_of_crit='training_day'))

    # Query days around the day at which criterion is reached
    days = (behavior_analyses.BehavioralSummaryByDate * subject.Subject * subj_crit_day
            & ('training_day - day_of_crit between %d and %d'
               % (-days_from_criterion[0], days_from_criterion[1]))).proj(
                   'subject_uuid', 'subject_nickname', 'session_date')

    # Use dates to query sessions
    ses_query = acquisition.Session.aggr(
                            days, from_date='min(session_date)', to_date='max(session_date)')

    sessions = (acquisition.Session * ses_query & 'date(session_start_time) >= from_date'
                & 'date(session_start_time) <= to_date')

    # Exclude weird sessions
    sessions = sessions & dj.Not([{'session_uuid': u_id} for u_id in EXCLUDED_SESSIONS])

    # Transform to pandas dataframe if necessary
    if as_dataframe is True:
        sessions = sessions.fetch(format='frame').reset_index()
        days = days.fetch(format='frame').reset_index()

    return sessions, days
def dj2pandas(behav):

    # make sure all contrasts are positive
    behav['trial_stim_contrast_right'] = np.abs(
        behav['trial_stim_contrast_right'])
    behav['trial_stim_contrast_left'] = np.abs(
        behav['trial_stim_contrast_left'])

    behav['signed_contrast'] = (
        behav['trial_stim_contrast_right'] - behav['trial_stim_contrast_left']) * 100
    # behav['signed_contrast'] = behav.signed_contrast.astype(int)

    behav['trial'] = behav.trial_id  # for psychfuncfit
    val_map = {'CCW': 1, 'No Go': 0, 'CW': -1}
    behav['choice'] = behav['trial_response_choice'].map(val_map)
    behav['correct'] = np.where(
        np.sign(behav['signed_contrast']) == behav['choice'], 1, 0)
    behav.loc[behav['signed_contrast'] == 0, 'correct'] = np.NaN

    behav['choice_right'] = behav.choice.replace(
        [-1, 0, 1], [0, np.nan, 1])  # code as 0, 100 for percentages
    behav['choice2'] = behav.choice_right  # for psychfuncfit
    behav['correct_easy'] = behav.correct
    behav.loc[np.abs(behav['signed_contrast']) < 50, 'correct_easy'] = np.NaN
    behav.rename(
        columns={'trial_stim_prob_left': 'probabilityLeft'}, inplace=True)
    behav['probabilityLeft'] = behav['probabilityLeft'] * 100
    behav['probabilityLeft'] = behav.probabilityLeft.astype(int)

    # compute rt
    if 'trial_response_time' in behav.columns:
        behav['rt'] = behav['trial_response_time'] - \
            behav['trial_stim_on_time']
        # ignore a bunch of things for missed trials
        # don't count RT if there was no response
        behav.loc[behav.choice == 0, 'rt'] = np.nan
        # don't count RT if there was no response
        behav.loc[behav.choice == 0, 'trial_feedback_type'] = np.nan

    # CODE FOR HISTORY
    behav['previous_choice'] = behav.choice.shift(1)
    behav.loc[behav.previous_choice == 0, 'previous_choice'] = np.nan
    behav['previous_outcome'] = behav.trial_feedback_type.shift(1)
    behav.loc[behav.previous_outcome == 0, 'previous_outcome'] = np.nan
    behav['previous_contrast'] = np.abs(behav.signed_contrast.shift(1))
    behav['previous_choice_name'] = behav['previous_choice'].map(
        {-1: 'left', 1: 'right'})
    behav['previous_outcome_name'] = behav['previous_outcome'].map(
        {-1: 'post_error', 1: 'post_correct'})
    behav['repeat'] = (behav.choice == behav.previous_choice)

    # # to more easily retrieve specific training days
    # behav['days'] = (behav['session_start_time'] -
    #                  behav['session_start_time'].min()).dt.days

    return behav
def institution_map():
    institution_map = {'UCL': 'Lab 1', 'CCU': 'Lab 2', 'CSHL': 'Lab 3', 'NYU': 'Lab 4',
                       'Princeton': 'Lab 5', 'SWC': 'Lab 6', 'Berkeley': 'Lab 7'}
    col_names = ['Lab 1', 'Lab 2', 'Lab 3', 'Lab 4', 'Lab 5', 'Lab 6', 'Lab 7', 'All labs']

    return institution_map, col_names

In [11]:
use_sessions, _ = query_sessions_around_criterion(criterion='biased',
                                                    days_from_criterion=[2, 3],
                                                    as_dataframe=False,
                                                    force_cutoff=False)

trial_fields = ('trial_stim_contrast_left', 'trial_stim_contrast_right',
                'trial_response_time', 'trial_stim_prob_left',
                'trial_feedback_type', 'trial_stim_on_time', 'trial_response_choice')

# query trial data for sessions and subject name and lab info
trials = use_sessions.proj('task_protocol') * behavior.TrialSet.Trial.proj(*trial_fields)
subject_info = subject.Subject.proj('subject_nickname') * \
                (subject.SubjectLab * reference.Lab).proj('institution_short')

# Fetch, join and sort data as a pandas DataFrame
behav = dj2pandas(trials.fetch(format='frame')
                            .join(subject_info.fetch(format='frame'))
                            .sort_values(by=['institution_short', 'subject_nickname',
                                            'session_start_time', 'trial_id'])
                            .reset_index())


In [12]:
#behav['institution_code'] = behav.institution_short.map(institution_map)
# split the two types of task protocols (remove the pybpod version number)
behav['task'] = behav['task_protocol'].str[14:20].copy()

# RECODE SOME THINGS JUST FOR PATSY
behav['contrast'] = np.abs(behav.signed_contrast)
behav['stimulus_side'] = np.sign(behav.signed_contrast)
behav['block_id'] = behav['probabilityLeft'].map({80:-1, 50:0, 20:1})

# hebav_dropna is a `pandas.DataFrame` already

In [13]:
behav_dropna = behav.dropna(subset=['stimulus_side','previous_choice','previous_outcome','choice'])

## 修改此处选取不同小鼠

In [14]:
idxs_subject = np.arange(31,65 + 1)
print(idxs_subject)

[31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54
 55 56 57 58 59 60 61 62 63 64 65]


# HDDM

In [15]:
# load data into hddm format and pick the necessary columns
data = behav_dropna[['choice', 'task','signed_contrast','subject_nickname', 'rt','previous_choice_name']]

In [16]:
stacked_data_for_fit = []
for idx_subject in idxs_subject:
    data_pick = data[np.logical_and(data['task']== 'traini', data['subject_nickname']== df_nickname.loc[idx_subject,'subject_nickname'])]
    data_for_fit = data_pick.copy()
    data_for_fit = data_for_fit[['rt','choice', 'signed_contrast','previous_choice_name']]
    # remove signed-contrast == 50
    data_for_fit = data_for_fit[np.abs(data_for_fit['signed_contrast'].values) != 50]
    data_for_fit['subj_idx'] = idx_subject
    data_for_fit = data_for_fit.rename(columns = {'rt':'rt','choice':'response','subj_idx':'subj_idx'})
    idx_rt_restrict = np.logical_and(data_for_fit['rt']> 0,
                                    data_for_fit['rt']< 10)
    data_for_fit = data_for_fit[idx_rt_restrict]
    stacked_data_for_fit.append(data_for_fit)
stacked_data_for_fit = pd.concat(stacked_data_for_fit)



In [17]:
stacked_data_for_fit

,rt,response,signed_contrast,previous_choice_name,subj_idx
120835,0.3451,1,100.00,left,33
120838,0.3397,1,100.00,left,33
120840,0.2526,-1,-100.00,right,33
120841,0.2574,1,100.00,left,33
120842,0.2293,1,100.00,right,33
...,...,...,...,...,...
475151,0.3862,-1,-100.00,right,65
475152,0.5366,1,6.25,left,65
475153,0.5838,1,12.50,right,65
475154,0.3933,1,12.50,right,65


In [18]:
############################################
# Instantiate model object passing it our data (no need to call flip_errors() before passing it).
# This will tailor an individual hierarchical DDM around your dataset.
m = hddm.HDDM(stacked_data_for_fit, depends_on = {'v':['signed_contrast','previous_choice_name']},
              is_group_model = True,
              bias = True, # include the bias term, so we will have {a,v,t,b} parameters
              )
# find a good starting point which helps with the convergence.
m.find_starting_values()
# start drawing 7000 samples and discarding 5000 as burn-in
m.sample(1000, burn=20)

/usr/local/lib/python3.7/dist-packages/scipy/optimize/optimize.py:1985: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)


 [                  0%                  ] 9 of 1000 complete in 20.0 secHalting at iteration  8  of  1000
Could not generate output statistics for v_subj(right.100.0).52
Could not generate output statistics for v_subj(right.-25.0).42
Could not generate output statistics for v_subj(left.-100.0).40
Could not generate output statistics for v_subj(right.0.0).34
Could not generate output statistics for v_subj(left.-12.5).64
Could not generate output statistics for v_subj(right.-6.25).48
Could not generate output statistics for t
Could not generate output statistics for t_subj.36
Could not generate output statistics for v_subj(left.25.0).50
Could not generate output statistics for z_subj_trans.53
Could not generate output statistics for z_subj_trans.61
Could not generate output statistics for v_subj(left.-100.0).36
Could not generate output statistics for v_subj(left.-6.25).38
Could not generate output statistics for z_subj_trans.65
Could not generate output statistics for v_subj(right.100.0

TypeError: ignored

In [ ]:
stats = m.gen_stats()
trace = m.get_traces()

# save
stats.to_csv("stats_0_10.csv",index = False)
trace.to_csv("trace_0_10.csv",index = False)